# Development

In [ ]:
%load_ext autoreload
%autoreload 2
# import skimage.io
# import matplotlib
# import matplotlib.pyplot as plt
# %matplotlib inline
# nd2 = "/home/dan/pims/pims/tests/data/bioformats/cluster.nd2"
from pathlib import Path

import nima_io.read as ir

tdata = Path("../../tests/data/")

lif = tdata / "2015Aug28_TransHXB2_50min+DMSO.lif"
img_tile = tdata / "t4_1.tif"  # C=3 T=4 S=15
img_void_tile = tdata / "tile6_1.tif"  # C=4 T=3 S=14 scattered
# imgsingle = tdata / "exp2_2.tif"  # C=2 T=81

# mcts = tdata / "multi-channel-time-series.ome.tif"  # C=3 T=7
# bigtiff = tdata / "LC26GFP_1.tf8"  # bigtiff

slif = str(lif)
simg_tile = str(img_tile)
simg_void_tile = str(img_void_tile)
# simgsingle = str(imgsingle)
# smcts = str(mcts)Z`Z
# sbigtiff = str(bigtiff)

Core metadata list at:
https://docs.openmicroscopy.org/bio-formats/7.1.0/developers/file-reader.html



In [ ]:
md, wr = ir.read(slif)

In [ ]:
wr.rdr.setSeries(4)
wr.rdr.getImageCount()

In [ ]:
wr.rdr.getDimensionOrder()

TODO: test convert value

In [ ]:
ome_store = wr.rdr.getMetadataStore()

## XML
getRoot was used in imgread-0.2.1

In [ ]:
root = ome_store.getRoot()
instrument = root.getInstrument(0)
detector = instrument.getDetector(0)
detector.getModel()

In [ ]:
wr.rdr.getMetadataStoreRoot().getInstrument(0).getDetector(0).getModel()

In [ ]:
wr.rdr.getMetadataStore().getRoot().getInstrument(0).getDetector(0).getModel()

In [ ]:
[m for m in root.__dir__() if m[:3] == "get"]

In [ ]:
image = root.getImage(3)
[m for m in image.__dir__() if m[:3] == "get"]

In [ ]:
obj = image.getObjectiveSettings()
[m for m in obj.__dir__() if m[:3] == "get"]
obj.getID()

In [ ]:
pixels = image.getPixels()
[m for m in pixels.__dir__() if m[:3] == "get"]

In [ ]:
ch = pixels.getChannel(0)
[m for m in ch.__dir__() if m[:3] == "get"]

In [ ]:
lp = ch.getLightPath()
[m for m in lp.__dir__() if m[:3] == "get"]

In [ ]:
pixels.getTimeIncrement()

In [ ]:
plane = pixels.getPlane(11)
[m for m in plane.__dir__() if m[:3] == "get"]

In [ ]:
import jpype

jpype.JObject(plane.getTheC()), plane.getDeltaT(), plane.getExposureTime()

In [ ]:
for k in root.__dir__():
    if k[:3] == "get":
        print(k)

In [ ]:
im = root.getImage(0)
obj = im.getObjectiveSettings()
(
    obj.getID(),
    obj.getMedium(),
    obj.getRefractiveIndex(),
    obj.getCorrectionCollar(),
    obj.getObjective(),
)

In [ ]:
(
    wr.rdr.getMetadataStore().getObjectiveID(0, 0) == obj.getID(),
    wr.rdr.getMetadataStore().getObjectiveLensNA(0, 0),
)

## md vs root (to remember)

from md i can get info on camera, objectives ...

but also key parameters like exposuretime, DeltaT, planePositionXYZ

In [ ]:
md_vtile, wr_vtile = ir.read(simg_void_tile)

In [ ]:
wr_vtile.rdr.getMetadataStore().getObjectiveCount(0)

In [ ]:
ome_store = wr_vtile.rdr.getMetadataStore()
ome_store.getXMLAnnotationValue(
    0
) == ome_store.getRoot().getStructuredAnnotations().getXMLAnnotation(0).getValue()

In [ ]:
# import bioformats

# md = bioformats.get_omexml_metadata(slif)
from xml.etree import ElementTree as ET

# mdroot = ET.fromstring(wr.rdr.getMetadataStore().dumpXML())
java_string = (
    wr_vtile.rdr.getMetadataStoreRoot()
    .getStructuredAnnotations()
    .getXMLAnnotation(0)
    .getValue()
)
python_string = jpype.java.lang.String(java_string).toString()
mdroot = ET.fromstring(python_string.getBytes())

for a in mdroot:
    print((a.tag, a.attrib))
    for aa in a:
        print((" GGSSS ", aa.tag, aa.attrib))
        for aaa in aa:
            print((" GGSSS2 ", aaa.tag, aaa.attrib))

In [ ]:
import xmltodict

m1 = xmltodict.parse(ome_store.dumpXML().getBytes())
list(m1["OME"]), m1["OME"]["Instrument"]

In [ ]:
metalist = [
    (child.tag, [[child.attrib]])
    for child in ET.fromstring(ome_store.dumpXML().getBytes()).iter()
]

In [ ]:
m = metalist[10]
m[0], m[1][0][0]

In [ ]:
for child in mdroot:
    if child.tag.endswith(""):
        for grandchild in child:
            print((grandchild.tag, grandchild.attrib))